In [1]:
using StatisticalRethinking, Distributed, JLD
using Mamba

# Data
line = Dict{Symbol, Any}()

howell1 = CSV.read(joinpath(dirname(Base.pathof(StatisticalRethinking)), "..", "data", "Howell1.csv"), delim=';')
df = convert(DataFrame, howell1);

Use only adults

In [2]:
df2 = filter(row -> row[:age] >= 18, df);
line[:x] = convert(Array{Float64,1}, df2[:weight]);
line[:y] = convert(Array{Float64,1}, df2[:height]);
line[:xmat] = convert(Array{Float64,2}, [ones(length(line[:x])) line[:x]])

# Model Specification
model = Model(
  y = Stochastic(1,
    (xmat, beta, s2) -> MvNormal(xmat * beta, sqrt(s2)),
    false
  ),
  beta = Stochastic(1, () -> MvNormal([178, 0], [sqrt(10000), sqrt(100)])),
  s2 = Stochastic(() -> Uniform(0, 50))
)


# Initial Values
inits = [
  Dict{Symbol, Any}(
    :y => line[:y],
    :beta => [rand(Normal(178, 100)), rand(Normal(0, 10))],
    :s2 => rand(Uniform(0, 50))
  )
  for i in 1:3
]

# Tuning Parameters
scale1 = [0.5, 0.25]
summary1 = identity
eps1 = 0.5

scale2 = 0.5
summary2 = x -> [mean(x); sqrt(var(x))]
eps2 = 0.1

# Define sampling scheme

scheme = [
  Mamba.NUTS([:beta]),
  Mamba.Slice([:s2], 10)
]

setsamplers!(model, scheme)

# MCMC Simulation

sim = mcmc(model, line, inits, 10000, burnin=1000, chains=3)

MCMC Simulation of 10000 Iterations x 3 Chains...

Chain 1:   0% [0:11:29 of 0:11:30 remaining]
Chain 1:  10% [0:00:36 of 0:00:40 remaining]
Chain 1:  20% [0:00:42 of 0:00:52 remaining]
Chain 1:  30% [0:00:41 of 0:00:58 remaining]
Chain 1:  40% [0:00:36 of 0:01:01 remaining]
Chain 1:  50% [0:00:31 of 0:01:01 remaining]
Chain 1:  60% [0:00:25 of 0:01:02 remaining]
Chain 1:  70% [0:00:19 of 0:01:03 remaining]
Chain 1:  80% [0:00:13 of 0:01:04 remaining]
Chain 1:  90% [0:00:06 of 0:01:04 remaining]
Chain 1: 100% [0:00:00 of 0:01:05 remaining]

Chain 2:   0% [0:00:02 of 0:00:02 remaining]
Chain 2:  10% [0:00:45 of 0:00:50 remaining]
Chain 2:  20% [0:00:45 of 0:00:56 remaining]
Chain 2:  30% [0:00:41 of 0:00:59 remaining]
Chain 2:  40% [0:00:36 of 0:00:59 remaining]
Chain 2:  50% [0:00:30 of 0:01:01 remaining]
Chain 2:  60% [0:00:25 of 0:01:02 remaining]
Chain 2:  70% [0:00:19 of 0:01:03 remaining]
Chain 2:  80% [0:00:13 of 0:01:03 remaining]
Chain 2:  90% [0:00:06 of 0:01:03 remaining]
Cha

Object of type "Mamba.ModelChains"

Iterations = 1001:10000
Thinning interval = 1
Chains = 1,2,3
Samples per chain = 9000

[24.8016 113.721 0.905323; 27.7189 113.714 0.916506; … ; 27.4063 114.829 0.879132; 27.373 114.829 0.879132]

[29.1617 115.022 0.888203; 29.3254 115.016 0.864627; … ; 32.0125 114.65 0.881944; 31.9451 114.516 0.884708]

[26.4146 113.063 0.921563; 25.9122 111.579 0.945858; … ; 24.4041 116.852 0.837413; 28.2536 113.904 0.900264]

Show draws summary

In [3]:
describe(sim)

Iterations = 1001:10000
Thinning interval = 1
Chains = 1,2,3
Samples per chain = 9000

Empirical Posterior Estimates:
            Mean          SD        Naive SE       MCSE        ESS   
     s2  26.16044225 2.002665267 0.01218783269 0.0193921196 9000.0000
beta[1] 113.85056121 1.902110038 0.01157587305 0.0358494066 2815.1875
beta[2]   0.90572963 0.041852046 0.00025470344 0.0007828625 2858.0012

Quantiles:
            2.5%       25.0%       50.0%       75.0%       97.5%   
     s2  22.5326098  24.7618047  26.0511718  27.4357015  30.4144890
beta[1] 110.0707718 112.5658912 113.8999698 115.1519273 117.5280554
beta[2]   0.8244975   0.8770842   0.9049083   0.9344114   0.9889133



End of `clip_38.0m.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*